In [4]:
import json
import xml.etree.ElementTree as ET
from pathlib import Path
from pydantic import BaseModel, validator, Field

datafile = Path("merry20210929154407.output.json")

# Read JSON file
with open(datafile, "r") as f:
    data = json.load(f)


class Effect(BaseModel):
    starttime: str = Field(alias="start_time")
    endtime: str = Field(alias="end_time")
    label: str = Field(alias="content")

    @validator("starttime", "endtime", pre=True)
    def check_time(cls, v):
        return str(int(float(v) * 1000))


class Effects(BaseModel):
    effects: list[Effect]


In [6]:
words = []
for i in data["results"]["items"]:
    if i["type"] != "pronunciation":
        continue

    if len(words) == 0:
            this_word = Effect(
                content="",
                start_time=0,
                end_time=i["start_time"],
            )
            words.append(this_word)
    elif words[-1].endtime != i["start_time"]:
            this_word = Effect(
                content="",
                start_time=int(words[-1].endtime) / 1000,
                end_time=i["start_time"],
            )
            words.append(this_word)


    this_word = Effect(
        content=i["alternatives"][0]["content"],
        start_time=i["start_time"],
        end_time=i["end_time"],
    )


    # print(this_word)
    # effect = Effect(**this_word)
    words.append(this_word)

In [3]:
len(words)

246

In [4]:
words[0]

Effect(starttime='0', endtime='240', content='')

In [5]:
words[-1]

Effect(start_time='198800', end_time='198940', content='huh')

In [12]:
effects = Effects(effects=words)

<?xml version="1.0" encoding="UTF-8"?>
<timings>
    <timing name="New Timing" SourceVersion="2018.4">
        <EffectLayer>
            <Effect label="It\'s" startTime="240" endTime="12270" />

In [7]:
root = ET.Element("timings")
timing = ET.SubElement(root, "timing", attrib={"name": "New Timing", "SourceVersion": "2018.4"})
effectlayer = ET.SubElement(timing, "EffectLayer")
for word in words:
    ET.SubElement(effectlayer, "Effect", attrib=word.dict())

In [9]:
ET.dump(root)

<timings><timing name="New Timing" SourceVersion="2018.4"><EffectLayer><Effect starttime="0" endtime="240" label="" /><Effect starttime="240" endtime="12270" label="It's" /><Effect starttime="12270" endtime="12270" label="" /><Effect starttime="12270" endtime="12400" label="the" /><Effect starttime="12400" endtime="12410" label="" /><Effect starttime="12410" endtime="12910" label="guts" /><Effect starttime="12910" endtime="12910" label="" /><Effect starttime="12910" endtime="13160" label="and" /><Effect starttime="13160" endtime="13160" label="" /><Effect starttime="13160" endtime="13390" label="it's" /><Effect starttime="13390" endtime="13390" label="" /><Effect starttime="13390" endtime="13560" label="the" /><Effect starttime="13560" endtime="13560" label="" /><Effect starttime="13560" endtime="14400" label="glory" /><Effect starttime="14400" endtime="14930" label="" /><Effect starttime="14930" endtime="15450" label="100" /><Effect starttime="15450" endtime="15450" label="" /><Effect